In [87]:
#######################################################################
###############           BRDF CORRECTION         #####################
#######################################################################
## Daniel Borini Alves; Dhemerson Conciani; and Swanni Alvarado

## --  adapted  based   on  Minh Nguyen  script   <https://github.com/ndminhhus/geeguide/blob/master/04.topo_correction.md>  and 
## Poortinga et al. (2018). Original background developed by Roy et al (2017) for Landsat data, and also successfully applied over 
## Sentinel 2 datasets (Zhang et al., 2018). Compute and correct Bidirectional Reflectance Distribution Function (BRDF) effects.
## See all references in the end of the script.

## Inputs: Bottom of Atmosphere (BOA) reflectance (PY6s), adjusted geometric (AROSICS), of each data collection (Landsat and Sentinel)
## Output: BOA reflectance, ajusted geometric and BRDF 

## ACTIVATE PACKAGES
import datetime
import math
import os
import sys
import ee

ee.Initialize()

In [88]:
## ACCESS IMAGE COLLECTIONS

# Define your time interval, centroid of study area and area of interest
startDate = ee.Date('2019-09-01')
endDate   = ee.Date('2019-10-01') #long periods can spend a lot of processing time...
geom = ee.Geometry.Point(-61.5775286643,-8.4852883834)
#region = geom.buffer(20000).bounds().getInfo()['coordinates']
region = ee.FeatureCollection("users/danielborini/parcelas_PNCA/Recorte_ZonasAeB_Sent2")

print ('Available images in the period:')
# Sentinel 2A/2B collection images
S2_col = ee.ImageCollection('COPERNICUS/S2').filterBounds(geom).filterDate(startDate,endDate)
col_length_S2 = S2_col.size().getInfo()
print ('S2A/2B images = ', col_length_S2)

# Landsat OLI collection images
L8_col = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterBounds(geom).filterDate(startDate,endDate)
col_length_L8 = L8_col.size().getInfo()
print ('L8 images = ', col_length_L8)

# Landsat ETM+ collection images
L7_col = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA').filterBounds(geom).filterDate(startDate,endDate)
col_length_L7 = L7_col.size().getInfo()
print ('L7 images = ', col_length_L7)

Available images in the period:
S2A/2B images =  5
L8 images =  1
L7 images =  2


In [89]:
## BAND ASSIGNATIONS AND BRDF BASIC PARAMETERS

# Input and output band assgnations definition
bandIn_L8  = ['B2','B3','B4','B5','B6','B7']
bandOut_L8 = ['blue','green','red','nir','swir1','swir2']

bandIn_L7  = ['B1','B2','B3','B4','B5','B7']
bandOut_L7 = ['blue','green','red','nir','swir1','swir2']

bandIn_S2  = ['B2','B3','B4','B8A','B11','B12']
bandOut_S2 = ['blue','green','red','nir','swir1','swir2']

## BRDF parameters
PI = ee.Number(3.14159265359)
MAX_SATELLITE_ZENITH = 7.5
MAX_DISTANCE = 1000000
UPPER_LEFT = 0
LOWER_LEFT = 1
LOWER_RIGHT = 2
UPPER_RIGHT = 3

In [84]:
## BRDF CORRECTION FUNCIONS DEFINITION

def applyBRDF_Landsat(image):
    #date = ee.Date('2019-08-05') ### 
    info = image.getInfo()['properties'] ##added
    date_fromimage = datetime.datetime.utcfromtimestamp(info['system:time_start']/1000) ##added
    date = ee.Date(date_fromimage.strftime('%Y-%m-%d')) ##added
    footprint = ee.List(image.geometry().bounds().bounds().coordinates().get(0))
    angles =  getsunAngles(date, footprint)
    sunAz = angles[0]
    sunZen = angles[1]
  
    viewAz = azimuth(footprint)
    viewZen = zenith(footprint)
  
  
    kval = _kvol(sunAz, sunZen, viewAz, viewZen)
    kvol = kval[0]
    kvol0 = kval[1]
    result = _apply_Landsat(image, kvol.multiply(PI), kvol0.multiply(PI))
  
    return result

def applyBRDF_Sentinel(image):
    #date = ee.Date('2018-11-07') #
    #date = image.date() #
    info = image.getInfo()['properties'] ##added
    date_fromimage = datetime.datetime.utcfromtimestamp(info['system:time_start']/1000) ##added
    footprint = ee.List(image.geometry().bounds().bounds().coordinates().get(0))
    angles =  getsunAngles(date, footprint)
    sunAz = angles[0]
    sunZen = angles[1]
  
    viewAz = azimuth(footprint)
    viewZen = zenith(footprint)
    
    kval = _kvol(sunAz, sunZen, viewAz, viewZen)
    kvol = kval[0]
    kvol0 = kval[1]
    result = _apply_Sentinel(image, kvol.multiply(PI), kvol0.multiply(PI))
  
    return result

def getsunAngles(date, footprint):
    jdp = date.getFraction('year')
    seconds_in_hour = 3600
    hourGMT = ee.Number(date.getRelative('second', 'day')).divide(seconds_in_hour)
    
    latRad = ee.Image.pixelLonLat().select('latitude').multiply(PI.divide(180))
    longDeg = ee.Image.pixelLonLat().select('longitude')
    
    # Julian day proportion in radians
    jdpr = jdp.multiply(PI).multiply(2)
    
    a = ee.List([0.000075, 0.001868, 0.032077, 0.014615, 0.040849])
    meanSolarTime = longDeg.divide(15.0).add(ee.Number(hourGMT))
    localSolarDiff1 = value(a, 0).add(value(a, 1).multiply(jdpr.cos())).subtract(value(a, 2).multiply(jdpr.sin())).subtract(value(a, 3).multiply(jdpr.multiply(2).cos())).subtract(value(a, 4).multiply(jdpr.multiply(2).sin()))
    
    localSolarDiff2 = localSolarDiff1.multiply(12 * 60)
  
    localSolarDiff = localSolarDiff2.divide(PI)
    trueSolarTime = meanSolarTime.add(localSolarDiff.divide(60)).subtract(12.0)
    
    # Hour as an angle
    ah = trueSolarTime.multiply(ee.Number(MAX_SATELLITE_ZENITH * 2).multiply(PI.divide(180)))
    b = ee.List([0.006918, 0.399912, 0.070257, 0.006758, 0.000907, 0.002697, 0.001480])
    delta = value(b, 0).subtract(value(b, 1).multiply(jdpr.cos())).add(value(b, 2).multiply(jdpr.sin())).subtract(value(b, 3).multiply(jdpr.multiply(2).cos())).add(value(b, 4).multiply(jdpr.multiply(2).sin())).subtract(value(b, 5).multiply(jdpr.multiply(3).cos())).add(value(b, 6).multiply(jdpr.multiply(3).sin()))

    cosSunZen = latRad.sin().multiply(delta.sin()).add(latRad.cos().multiply(ah.cos()).multiply(delta.cos()))
    sunZen = cosSunZen.acos()

    # sun azimuth from south, turning west
    sinSunAzSW = ah.sin().multiply(delta.cos()).divide(sunZen.sin())
    sinSunAzSW = sinSunAzSW.clamp(-1.0, 1.0)
  
    cosSunAzSW = (latRad.cos().multiply(-1).multiply(delta.sin()).add(latRad.sin().multiply(delta.cos()).multiply(ah.cos()))).divide(sunZen.sin())
    sunAzSW = sinSunAzSW.asin()
  
    sunAzSW = where(cosSunAzSW.lte(0), sunAzSW.multiply(-1).add(PI), sunAzSW)
    sunAzSW = where(cosSunAzSW.gt(0).And(sinSunAzSW.lte(0)), sunAzSW.add(PI.multiply(2)), sunAzSW) # changed .and to .And
  
    sunAz = sunAzSW.add(PI)
    # Keep within [0, 2pi] range
    sunAz = where(sunAz.gt(PI.multiply(2)), sunAz.subtract(PI.multiply(2)), sunAz)
  
    footprint_polygon = ee.Geometry.Polygon(footprint)
    sunAz = sunAz.clip(footprint_polygon)
    sunAz = sunAz.rename(['sunAz'])
    sunZen = sunZen.clip(footprint_polygon).rename(['sunZen'])
  
    return [sunAz, sunZen]

def azimuth(footprint):
    def x (point):return ee.Number(ee.List(point).get(0))
    def y (point):return ee.Number(ee.List(point).get(1))
    
    upperCenter = line_from_coords(footprint, UPPER_LEFT, UPPER_RIGHT).centroid().coordinates()
    lowerCenter = line_from_coords(footprint, LOWER_LEFT, LOWER_RIGHT).centroid().coordinates()
    slope = ((y(lowerCenter)).subtract(y(upperCenter))).divide((x(lowerCenter)).subtract(x(upperCenter)))
    slopePerp = ee.Number(-1).divide(slope);
    azimuthLeft = ee.Image(PI.divide(2).subtract((slopePerp).atan()))
    
    return azimuthLeft.rename(['viewAz'])

def zenith(footprint):
    leftLine = line_from_coords(footprint, UPPER_LEFT, LOWER_LEFT)
    rightLine = line_from_coords(footprint, UPPER_RIGHT, LOWER_RIGHT)
    leftDistance = ee.FeatureCollection(leftLine).distance(MAX_DISTANCE)
    rightDistance = ee.FeatureCollection(rightLine).distance(MAX_DISTANCE)
    viewZenith = rightDistance.multiply(ee.Number(MAX_SATELLITE_ZENITH * 2)).divide(rightDistance.add(leftDistance)).subtract(ee.Number(MAX_SATELLITE_ZENITH)).clip(ee.Geometry.Polygon(footprint)).rename(['viewZen'])
    
    return viewZenith.multiply(PI.divide(180))

def _apply_Sentinel(image, kvol, kvol0):
    f_iso = 0
    f_geo = 0
    f_vol = 0
    
    blue = _correct_band(image, 'blue', kvol, kvol0, f_iso=0.0774, f_geo=0.0079, f_vol=0.0372)
    green = _correct_band(image, 'green', kvol, kvol0, f_iso=0.1306, f_geo=0.0178, f_vol=0.0580)
    red = _correct_band(image, 'red', kvol, kvol0, f_iso=0.1690, f_geo=0.0227, f_vol=0.0574)
    re1 = _correct_band(image, 're1', kvol, kvol0, f_iso=0.2085, f_geo=0.0256, f_vol=0.0845)
    re2 = _correct_band(image, 're2', kvol, kvol0, f_iso=0.2316, f_geo=0.0273, f_vol=0.1003)
    re3 = _correct_band(image, 're3', kvol, kvol0, f_iso=0.2599, f_geo=0.0294, f_vol=0.1197)
    nir = _correct_band(image, 'nir', kvol, kvol0, f_iso=0.3093, f_geo=0.0330, f_vol=0.1535)
    re4 = _correct_band(image, 're4', kvol, kvol0, f_iso=0.2907, f_geo=0.0410, f_vol=0.1611)
    swir1 = _correct_band(image, 'swir1', kvol, kvol0, f_iso=0.3430, f_geo=0.0453, f_vol=0.1154) 
    swir2 = _correct_band(image, 'swir2', kvol, kvol0, f_iso=0.2658, f_geo=0.0387, f_vol=0.0639)
        
    return image.select([]).addBands([blue, green, red, nir,re1,re2,re3,nir,re4,swir1, swir2])

def _apply_Landsat(image, kvol, kvol0):
    f_iso = 0
    f_geo = 0
    f_vol = 0
    
    blue = _correct_band(image, 'blue', kvol, kvol0, f_iso=0.0774, f_geo=0.0079, f_vol=0.0372)
    green = _correct_band(image, 'green', kvol, kvol0, f_iso=0.1306, f_geo=0.0178, f_vol=0.0580)
    red = _correct_band(image, 'red', kvol, kvol0, f_iso=0.1690, f_geo=0.0227, f_vol=0.0574)
    nir = _correct_band(image, 'nir', kvol, kvol0, f_iso=0.3093, f_geo=0.0330, f_vol=0.1535)
    swir1 = _correct_band(image, 'swir1', kvol, kvol0, f_iso=0.3430, f_geo=0.0453, f_vol=0.1154)   
    swir2 = _correct_band(image, 'swir2', kvol, kvol0, f_iso=0.2658, f_geo=0.0387, f_vol=0.0639)
        
    return image.select([]).addBands([blue, green, red, nir, swir1, swir2])

def _correct_band(image, band_name, kvol, kvol0, f_iso, f_geo, f_vol):
    ##"""fiso + fvol * kvol + fgeo * kgeo"""
    iso = ee.Image(f_iso)
    geo = ee.Image(f_geo)
    vol = ee.Image(f_vol)
    pred = vol.multiply(kvol).add(geo.multiply(kvol)).add(iso).rename(['pred'])
    pred0 = vol.multiply(kvol0).add(geo.multiply(kvol0)).add(iso).rename(['pred0'])
    cfac = pred0.divide(pred).rename(['cfac'])
    corr = image.select(band_name).multiply(cfac).rename([band_name])
    
    return corr

def _kvol(sunAz, sunZen, viewAz, viewZen):
    ##"""Calculate kvol kernel.
    ##From Lucht et al. 2000
    ##Phase angle = cos(solar zenith) cos(view zenith) + sin(solar zenith) sin(view zenith) cos(relative azimuth)"""

    relative_azimuth = sunAz.subtract(viewAz).rename(['relAz'])
    pa1 = viewZen.cos().multiply(sunZen.cos())
    pa2 = viewZen.sin().multiply(sunZen.sin()).multiply(relative_azimuth.cos())
    phase_angle1 = pa1.add(pa2)
    phase_angle = phase_angle1.acos()
    p1 = ee.Image(PI.divide(2)).subtract(phase_angle)
    p2 = p1.multiply(phase_angle1)
    p3 = p2.add(phase_angle.sin())
    p4 = sunZen.cos().add(viewZen.cos())
    p5 = ee.Image(PI.divide(4))

    kvol = p3.divide(p4).subtract(p5).rename(['kvol'])

    viewZen0 = ee.Image(0)
    pa10 = viewZen0.cos().multiply(sunZen.cos())
    pa20 = viewZen0.sin().multiply(sunZen.sin()).multiply(relative_azimuth.cos())
    phase_angle10 = pa10.add(pa20)
    phase_angle0 = phase_angle10.acos()
    p10 = ee.Image(PI.divide(2)).subtract(phase_angle0)
    p20 = p10.multiply(phase_angle10)
    p30 = p20.add(phase_angle0.sin())
    p40 = sunZen.cos().add(viewZen0.cos())
    p50 = ee.Image(PI.divide(4))

    kvol0 = p30.divide(p40).subtract(p50).rename(['kvol0'])

    return [kvol, kvol0]

def line_from_coords(coordinates, fromIndex, toIndex):
    return ee.Geometry.LineString(ee.List([
      coordinates.get(fromIndex),
      coordinates.get(toIndex)]))

def where(condition, trueValue, falseValue):
    trueMasked = trueValue.mask(condition)
    falseMasked = falseValue.mask(invertMask(condition))
    
    return trueMasked.unmask(falseMasked)

def invertMask(mask):
    return mask.multiply(-1).add(1)

def value(list,index):
    return ee.Number(list.get(index))

In [85]:
# APPLY BRDF CORRECTIONS TO LANDSAT 8 COLLECTION
for i in range (0,col_length_L8):
       
    L8_list = L8_col.toList(col_length_L8)
    image = ee.Image(L8_list.get(i)).select(bandIn_L8,bandOut_L8)
    fname = ee.String(image.get('system:index')).getInfo()
    fname = str(fname [12:20]) + '_L08'
    
    image_BOA_BRDF = applyBRDF_Landsat(image)
    task = ee.batch.Export.image.toDrive(**{
        'image': image_BOA_BRDF,
        'description':fname + '_BOA_BRDF',
        'folder':'P6s_export',
        'scale': 30,
        'fileFormat': 'GeoTIFF',
        'region': region.geometry().bounds(),
        'maxPixels': 1e13
    })

    task.start()
    print('exporting-->done')


exporting-->done


In [90]:
# APPLY BRDF CORRECTIONS TO LANDSAT 7 COLLECTION
for i in range (0,col_length_L7):
       
    L7_list = L7_col.toList(col_length_L7)
    image = ee.Image(L7_list.get(i)).select(bandIn_L7,bandOut_L7)
    fname = ee.String(image.get('system:index')).getInfo()
    fname = str(fname [12:20]) + '_L07'
    
    image_BOA_BRDF = applyBRDF_Landsat(image)
    task = ee.batch.Export.image.toDrive(**{
        'image': image_BOA_BRDF,
        'description':fname + '_BOA_BRDF',
        'folder':'P6s_export',
        'scale': 30,
        'fileFormat': 'GeoTIFF',
        'region': region.geometry().bounds(),
        'maxPixels': 1e13
    })

    task.start()
    print('exporting ' + fname + '--->done')

exporting-->done
exporting-->done


In [92]:
# APPLY BRDF CORRECTIONS TO SENTINEL 2 COLLECTION
for i in range (0,col_length_S2):
       
    S2_list = S2_col.toList(col_length_S2)
    image = ee.Image(S2_list.get(i)).select(bandIn_S2,bandOut_S2)
    fname = image.get('DATASTRIP_ID').getInfo()
    fname = str(fname [25:33]) + '_' + str(fname [0:3])
    
    image_BOA_BRDF = applyBRDF_Sentinel(image)
    task = ee.batch.Export.image.toDrive(**{
        'image': image_BOA_BRDF,
        'description':fname + '_BOA_BRDF',
        'folder':'P6s_export',
        'scale': 20,
        'fileFormat': 'GeoTIFF',
        'region': region.geometry().bounds(),
        'maxPixels': 1e13
    })

    task.start()
    print('exporting ' + fname + '--->done')

exporting-->done
exporting-->done
exporting-->done
exporting-->done
exporting-->done


In [ ]:
## REFERENCES ##

# -- BRDF functions
# ROY, D. P.; ZHANG, H. K.; JU, J.; GOMEZ-DANS, J. L.; LEWIS, P. E.; SCHAAF, C. B.; SUN, Q.; LI, J.; HUANG, H.; KOVALSKYY, V. A general method 
# to normalize Landsat reflectance data to nadir BRDF adjusted reflectance. Remote Sensing of Environment, v. 176, p. 255–271, 2016. 
# ROY, D. P.; LI, J.; ZHANG, H. K.; YAN, L.; HUANG, H.; LI, Z. Examination of Sentinel-2A multi-spectral instrument (MSI) reflectance anisotropy 
# and the suitability of a general method to normalize MSI reflectance to nadir BRDF adjusted reflectance. Remote Sensing of Environment, 
# v. 199, p. 25–38, set. 2017.
# ZHANG, H. K.; ROY, D. P.; YAN, L.; LI, Z.; HUANG, H.; VERMOTE, E.; SKAKUN, S.; ROGER, J.-C. Characterization of Sentinel-2A and Landsat-8 top 
# of atmosphere, surface, and nadir BRDF adjusted reflectance and NDVI differences. Remote Sensing of Environment, v. 215, p. 482–494,  2018.

# -- Harmonization of Landsat-Sentinel2 examples / scripts examples used here
# NGUYEN, M. D.; BAEZ-VILLANUEVA, O. M.; BUI, D. D.; NGUYEN, P. T.; RIBBE, L. Harmonization of Landsat and Sentinel 2 for crop monitoring 
# in drought prone areas: Case studies of Ninh Thuan (Vietnam) and Bekaa (Lebanon). Remote Sensing, v. 12, n. 2, p. 1–18, 2020.
# POORTINGA, A.; TENNESON, K.; SHAPIRO, A.; NQUYEN, Q.; SAN AUNG, K.; CHISHTIE, F.; SAAH, D. Mapping Plantations in Myanmar by Fusing 
# Landsat-8, Sentinel-2 and Sentinel-1 Data along with Systematic Error Quantification. Remote Sensing, v. 11, n. 7, p. 831, 2019.